# Doublet detection of scRNA-seq data
**Author**: Adam Klie (last modified: 10/08/2023)<br>
***
**Description**: This script performs doublet detection with scrublet abd scDoubletFinder. The input should be an h5ad file that can be read in using the read_h5ad command from ScanPy. The output is an h5ad file with doublets detected and removed.

In [21]:
# Imports
import os
import numpy as np
import scanpy as sc
import scanpy.external as sce
import matplotlib.pyplot as plt
import seaborn as sns

sc.settings.verbosity = 0
sc.settings.set_figure_params(
    dpi=80,
    facecolor="white",
    frameon=False,
)

# CellCommander
from cellcommander.detect_doublets import scrublet

import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


/tmp/ipykernel_868022/341209908.py:27: DeprecationWarning: The global conversion available with activate() is deprecated and will be removed in the next major release. Use a local converter.
  anndata2ri.activate()


In [2]:
# Paths
sample = "H1-D11"
input_h5ad_path = f"/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_scRNA-seq/annotation/26Oct23/scanpy/{sample}/threshold_qc/threshold_qc.h5ad"
outdir_path = f"/cellar/users/aklie/data/datasets/Zhu2023_sc-islet_scRNA-seq/annotation/26Oct23/scanpy/{sample}/detect_doublets"

In [12]:
# Define run parameters
output_prefix = "scrublet_only"
initial_clust_n_neighbors = 30
iniitial_clust_resolution = 0.5

In [9]:
# If output directory does not exist, create it.
if not os.path.exists(outdir_path):
    os.makedirs(outdir_path)

In [6]:
# Read in h5ad
adata = sc.read_h5ad(input_h5ad_path)
adata

AnnData object with n_obs × n_vars = 9210 × 36601
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'outlier', 'mt_outlier'
    var: 'gene_ids', 'feature_types', 'genome', 'mt', 'ribo', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

In [7]:
# Run the scrublet method
scrublet.run_scrublet(adata=adata)

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby(

Automatically set threshold at doublet score = 0.19
Detected doublet rate = 4.6%
Estimated detectable doublet fraction = 52.7%
Overall doublet rate:
	Expected   = 5.0%
	Estimated  = 8.7%


In [10]:
# Plot score distribution
scrublet.plot_scrublet(adata=adata, outdir_path=outdir_path)

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/external/pl.py:419: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  for idx, (batch_key, sub_obs) in enumerate(adata.obs.groupby(batches)):


In [11]:
# Save results
scrublet.save_scrublet(adata=adata, outdir_path=outdir_path)

In [17]:
# Add filter col
adata.obs["doublet_filter"] = adata.obs["scrublet_predicted_doublet"]

In [18]:
# Run a quick pipeline
adata_pp = adata.copy()
sc.pp.filter_genes(adata_pp, min_cells=20)
sc.pp.normalize_total(adata_pp)
sc.pp.log1p(adata_pp)
sc.pp.highly_variable_genes(adata_pp, n_top_genes=3000)
adata_pp = adata_pp[:, adata_pp.var.highly_variable]
sc.pp.pca(adata_pp)
sc.pp.neighbors(adata_pp, n_neighbors=initial_clust_n_neighbors)
sc.tl.umap(adata_pp)
sc.tl.leiden(adata_pp, resolution=iniitial_clust_resolution)

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_cate

In [22]:
# Plot doublet scores on UMAP
doublet_score_columns = [c for c in adata.obs.columns if "_score" in c]
adata_pp.obs[doublet_score_columns + ["doublet_filter"]] = adata.obs[doublet_score_columns + ["doublet_filter"]]
adata_pp.obs["doublet_filter"] = adata.obs["doublet_filter"].astype("category")
adata.obs["pre_doublet_filter_leiden"] = adata_pp.obs["leiden"]
with plt.rc_context():
    sc.pl.umap(
        adata_pp,
        color=["leiden"] + doublet_score_columns + ["doublet_filter"],
        vmin=0,
        vmax="p99", 
        sort_order=False, 
        frameon=False,
        cmap="Reds", 
    )
    plt.savefig(os.path.join(outdir_path, "doublet_scores_umap.png"))
    plt.close()

/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:1207: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(values):
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:1216: FutureWarning: The default value of 'ignore' for the `na_action` parameter in pandas.Categorical.map is deprecated and will be changed to 'None' in a future version. Please set na_action to the desired value to avoid seeing this warning
  color_vector = pd.Categorical(values.map(color_map))
/cellar/users/aklie/opt/miniconda3/envs/scverse-py39-R431/lib/python3.9/site-packages/scanpy/plotting/_tools/scatterplots.py:391: UserWarning: No data for colormapping provided via 'c'. Parameters 'cmap' will be ignored
  cax = scatter(
/cellar/users/aklie/opt/minicon

# DONE!

---

# Scratch